# Imports

In [1]:
import numpy as np
import pandas as pd
import matplotlib
from xgboost import XGBRegressor
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_absolute_error
import optuna
import cupy as cp

rng = np.random.RandomState(42)

# Data

In [2]:
data_path = 'https://raw.githubusercontent.com/antbartash/max_temp/master/data/data_features.csv'
data = pd.read_csv(data_path)
data['DATE'] = data['DATE'].astype('datetime64[ns]')

X_train = data.loc[data['DATE'].dt.year <= 2021].drop(columns=['TARGET', 'DATE']).copy()
y_train = data.loc[data['DATE'].dt.year <= 2021, 'TARGET'].copy()
X_valid = data.loc[data['DATE'].dt.year == 2022].drop(columns=['TARGET', 'DATE']).copy()
y_valid = data.loc[data['DATE'].dt.year == 2022, 'TARGET'].copy()
X_test = data.loc[data['DATE'].dt.year == 2023].drop(columns=['TARGET', 'DATE']).copy()
y_test = data.loc[data['DATE'].dt.year == 2023, 'TARGET'].copy()

X_train = cp.array(X_train)
y_train = cp.array(y_train)
X_valid = cp.array(X_valid)
y_valid = cp.array(y_valid)
X_test = cp.array(X_test)
y_test = cp.array(y_test)

print(f'Train: {X_train.shape}, {y_train.shape}')
print(f'Valid: {X_valid.shape}, {y_valid.shape}')
print(f'Test: {X_test.shape}, {y_test.shape}')

Train: (34938, 52), (34938,)
Valid: (2920, 52), (2920,)
Test: (2920, 52), (2920,)


# Baseline

In [3]:
model = XGBRegressor(n_jobs=-1, random_state=rng, device='cuda')
model.fit(X_train, y_train)

print(f'Train MAE: {mean_absolute_error(y_train.get(), model.predict(X_train))}')
print(f'Valid MAE: {mean_absolute_error(y_valid.get(), model.predict(X_valid))}')

Train MAE: 2.2857497390403356
Valid MAE: 3.2355165774854893


# Hyperparameter tuning

In [4]:
def load_data():
    data_path = 'https://raw.githubusercontent.com/antbartash/max_temp/master/data/data_features.csv'
    data = pd.read_csv(data_path)
    data['DATE'] = data['DATE'].astype('datetime64[ns]')
    X_train = data.loc[data['DATE'].dt.year <= 2021].drop(columns=['TARGET', 'DATE']).copy()
    y_train = data.loc[data['DATE'].dt.year <= 2021, 'TARGET'].copy()
    X_valid = data.loc[data['DATE'].dt.year == 2022].drop(columns=['TARGET', 'DATE']).copy()
    y_valid = data.loc[data['DATE'].dt.year == 2022, 'TARGET'].copy()
    X_train, y_train = cp.array(X_train), cp.array(y_train)
    X_valid, y_valid = cp.array(X_valid), cp.array(y_valid)
    return X_train, X_valid, y_train, y_valid

def objective(trial):
    PARAMS = {
        'n_estimators': trial.suggest_int('n_estimators', 50, 1000),
        'learning_rate': trial.suggest_float('learning_rate', 1e-6, 0.5),
        'max_depth': trial.suggest_int('max_depth', 1, 12),
        'gamma': trial.suggest_float('gamma', 0.0, 3.0),
        'reg_lambda': trial.suggest_float('reg_lambda', 0.0, 3.0),
        'subsample': trial.suggest_float('subsample', 0.3, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.3, 1.0)
        
    }
    X_train, _, y_train, _ = load_data()
    model = XGBRegressor(n_jobs=-1, device='cuda', random_state=rng, verbosity=0)
    model.set_params(**PARAMS)
    score = np.mean(cross_val_score(model, X_train.get(), y_train.get(), cv=3, scoring='neg_mean_absolute_error'))
    return score

In [5]:
sampler = optuna.samplers.TPESampler(
    n_startup_trials = 50, # the random sampling is used instead of the TPE algorithm until the given number of trials finish in the same study
    n_ei_candidates = 24, # number of candidate samples used to calculate the expected improvement.
    multivariate = True, # multivariate TPE when suggesting candidates; default: False
    seed = 42
)
study = optuna.create_study(
    pruner=optuna.pruners.MedianPruner(n_startup_trials=10, n_warmup_steps=100, interval_steps=20),
    direction='maximize',
    sampler=sampler,
    storage='sqlite:///db.sqlite3'
)
study.optimize(
    objective, n_trials = 1000,
    timeout = 3600*10, # in seconds
    n_jobs = 1,
    show_progress_bar = True
)

/usr/local/lib/python3.10/dist-packages/optuna/_experimental.py:31: ExperimentalWarning: Argument ``multivariate`` is an experimental feature. The interface can change in the future.
  warnings.warn(
[I 2024-12-19 09:14:57,318] A new study created in RDB with name: no-name-036458a2-cd95-485c-98d5-0fd8f07bc3cf


  0%|          | 0/1000 [00:00<?, ?it/s]

[I 2024-12-19 09:15:03,074] Trial 0 finished with value: -4.519017195206174 and parameters: {'n_estimators': 406, 'learning_rate': 0.47535720249065166, 'max_depth': 9, 'gamma': 1.7959754525911098, 'reg_lambda': 0.46805592132730955, 'subsample': 0.40919616423534183, 'colsample_bytree': 0.3406585285177396}. Best is trial 0 with value: -4.519017195206174.
[I 2024-12-19 09:15:11,539] Trial 1 finished with value: -3.335525273264045 and parameters: {'n_estimators': 873, 'learning_rate': 0.30055790475659266, 'max_depth': 9, 'gamma': 0.06175348288740734, 'reg_lambda': 2.909729556485983, 'subsample': 0.8827098485602951, 'colsample_bytree': 0.44863737747479326}. Best is trial 1 with value: -3.335525273264045.
[I 2024-12-19 09:15:13,199] Trial 2 finished with value: -2.9934577329913927 and parameters: {'n_estimators': 222, 'learning_rate': 0.09170307152220707, 'max_depth': 4, 'gamma': 1.5742692948967134, 'reg_lambda': 1.2958350559263474, 'subsample': 0.5038603981386294, 'colsample_bytree': 0.7282

In [6]:
print(f"Best score: {study.best_trial.value}")
print(f"Best params: {study.best_trial.params}")

Best score: -2.9719317917286325
Best params: {'n_estimators': 739, 'learning_rate': 0.01700312566894861, 'max_depth': 4, 'gamma': 2.7463444775190746, 'reg_lambda': 0.650988330057499, 'subsample': 0.32115031803156807, 'colsample_bytree': 0.9899998705384223}


In [7]:
model = XGBRegressor(
    n_estimators=739, learning_rate=0.017,
    max_depth=4, gamma=2.745, reg_lambda=0.651,
    subsample=0.321, colsample_bytree=0.99,
    device='cuda', n_jobs=-1, random_state=rng
)
model.fit(X_train, y_train)

print(f'Train MAE: {mean_absolute_error(y_train.get(), model.predict(X_train))}')
print(f'Valid MAE: {mean_absolute_error(y_valid.get(), model.predict(X_valid))}')

Train MAE: 2.8103391158136533
Valid MAE: 3.15147869351792


In [8]:
model = XGBRegressor(
    n_estimators=750, learning_rate=0.02,
    max_depth=4, gamma=2.75, reg_lambda=0.7,
    subsample=0.3, colsample_bytree=1,
    device='cuda', n_jobs=-1, random_state=rng
)
model.fit(X_train, y_train)

print(f'Train MAE: {mean_absolute_error(y_train.get(), model.predict(X_train))}')
print(f'Valid MAE: {mean_absolute_error(y_valid.get(), model.predict(X_valid))}')

Train MAE: 2.7942125773824302
Valid MAE: 3.1482519331370318
